# Analyse de Contrat Perpétuel vs Spot pour Position Longue


Les contrats dérivés occupent une place centrale en finance, permettant aux investisseurs de se couvrir contre les risques ou de spéculer sur les fluctuations futures des prix des actifs sous-jacents. Parmi ces instruments financiers, le contrat perpétuel émerge comme une innovation significative, particulièrement dans le domaine des cryptomonnaies. Contrairement aux contrats à terme traditionnels, qui possèdent une date d'expiration fixe, le contrat perpétuel n'a pas de date d'échéance, ce qui permet aux traders de maintenir une position ouverte indéfiniment. Cette caractéristique unique facilite une correspondance plus étroite entre les prix des contrats perpétuels et ceux du marché au comptant (spot), grâce à l'ajustement périodique des taux de financement. Ces taux visent à équilibrer la différence de prix entre le contrat perpétuel et le marché spot, reflétant ainsi la dynamique de l'offre et de la demande sans nécessiter de règlement physique de l'actif.

L'objectif de ce projet est d'examiner et de comparer les coûts associés à la détention d'une position longue sur un contrat perpétuel par rapport à l'achat direct d'un actif sur le marché spot.

## 1/ Collecte de données





In [ ]:
import requests
from datetime import datetime

# Convertir une date en timestamp (millisecondes)
def date_to_milliseconds(date_str):
    """Convertit une date au format AAAA-MM-JJ en millisecondes depuis l'époque UNIX."""
    epoch = datetime.utcfromtimestamp(0)  # Début de l'époque UNIX
    my_date = datetime.strptime(date_str, "%Y-%m-%d")
    delta = my_date - epoch
    return int(delta.total_seconds() * 1000.0)

start_time = date_to_milliseconds("2023-11-01")
end_time = date_to_milliseconds("2024-03-20")

symbol = 'BTCUSDT'
interval = '1d'  # Interval d'un jour
limit = 1000  # Nombre maximum de points de données par requête

def fetch_klines(url, symbol, interval, start_time, end_time, limit=500):
    data = []

    params = {
        'symbol': symbol,
        'interval': interval,
        'startTime': start_time,
        'endTime': end_time,
        'limit': limit
    }
    response = requests.get(url, params=params)
    segment = response.json()
    data.extend(segment)
    return data

spot_data = fetch_klines("https://api.binance.com/api/v3/klines", symbol, interval, start_time, end_time, limit)
perp_data = fetch_klines("https://fapi.binance.com/fapi/v1/klines", symbol, interval, start_time, end_time, limit)
funding_rate = fetch_klines("https://fapi.binance.com/fapi/v1/fundingRate", symbol, interval, start_time, end_time, limit)

Dans le cadre de notre projet, nous avons collecté des données historiques sur les prix du Bitcoin (BTC) par rapport à l'USDT (Tether) via l'API de Binance pour la période allant du 1er novembre 2023 au 20 mars 2024.

Une étape clé de notre démarche a été l'utilisation d'une fonction spécifique pour convertir les dates en millisecondes depuis l'époque UNIX, format requis par l'API de Binance pour les requêtes de données.

## 2/ Traitement des données et analyse

In [ ]:
import pandas as pd

# Noms des colonnes pour les DataFrame

columns = ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Asset Volume', 'Number of Trades', 'Taker Buy Base Volume', 'Taker Buy Quote Volume', 'Ignore']

# Conversion en DataFrame

df_spot = pd.DataFrame(spot_data, columns=columns)
df_perp = pd.DataFrame(perp_data, columns=columns)
df_funding = pd.DataFrame(funding_rate)

# Convertir les timestamps en dates lisibles

df_spot['Open Time'] = pd.to_datetime(df_spot['Open Time'], unit='ms')
df_spot['Close Time'] = pd.to_datetime(df_spot['Close Time'], unit='ms')
df_perp['Open Time'] = pd.to_datetime(df_perp['Open Time'], unit='ms')
df_perp['Close Time'] = pd.to_datetime(df_perp['Close Time'], unit='ms')
df_funding['fundingTime'] = pd.to_datetime(df_funding['fundingTime'], unit='ms')

# S'assurer que les colonnes numériques sont de type correct

columns_to_convert = ['Open', 'High', 'Low', 'Close', 'Volume', 'Quote Asset Volume', 'Number of Trades', 'Taker Buy Base Volume', 'Taker Buy Quote Volume']
df_perp[columns_to_convert] = df_perp[columns_to_convert].apply(pd.to_numeric)
df_spot[columns_to_convert] = df_spot[columns_to_convert].apply(pd.to_numeric)

columns_to_convert = ['fundingRate', 'markPrice']
df_funding[columns_to_convert] = df_funding[columns_to_convert].apply(pd.to_numeric)


In [ ]:
import numpy as np
import plotly.graph_objects as go

# Calculer les rendements quotidiens pour le spot et le perpétuel
df_spot['Daily Return'] = df_spot['Close'].pct_change()
df_perp['Daily Return'] = df_perp['Close'].pct_change()

# Statistiques descriptives des rendements
print("Statistiques descriptives pour le marché Spot:")
print(df_spot['Daily Return'].describe())
print("\nStatistiques descriptives pour le marché Perpétuel:")
print(df_perp['Daily Return'].describe())

# Volatilité (écart-type des rendements quotidiens)
volatility_spot = df_spot['Daily Return'].std()
volatility_perp = df_perp['Daily Return'].std()
print(f"\nVolatilité Spot: {volatility_spot}, Volatilité Perpétuelle: {volatility_perp}")

# Corrélation entre les prix de clôture spot et perpétuels
correlation = df_spot['Close'].corr(df_perp['Close'])
print(f"\nCorrélation entre les prix de clôture Spot et Perpétuel: {correlation}")


# Plot des prix de clôture Spot et Perpétuel

# Création du graphique
fig = go.Figure()

# Ajout de la trace pour les données Spot
fig.add_trace(go.Scatter(x=df_spot['Close Time'], y=df_spot['Close'],
                         mode='lines', name='Spot Close Price'))

# Ajout de la trace pour les données Perpétuelles
fig.add_trace(go.Scatter(x=df_perp['Close Time'], y=df_perp['Close'],
                         mode='lines', name='Perpetual Close Price',
                         line=dict(dash='dash')))

# Mise à jour du layout pour ajouter des titres et activer le zoom
fig.update_layout(title='Comparaison des Prix de Clôture Spot vs Perpétuel',
                  xaxis_title='Date',
                  yaxis_title='Prix de Clôture',
                  dragmode='zoom',  # Activer le zoom
                  hovermode='x unified')  # Afficher les informations au survol

fig.show()

Statistiques descriptives pour le marché Spot:
count    140.000000
mean       0.005057
std        0.028624
min       -0.083902
25%       -0.008393
50%        0.002746
75%        0.018297
max        0.095308
Name: Daily Return, dtype: float64

Statistiques descriptives pour le marché Perpétuel:
count    140.000000
mean       0.005060
std        0.028698
min       -0.084036
25%       -0.008386
50%        0.002589
75%        0.018437
max        0.095238
Name: Daily Return, dtype: float64

Volatilité Spot: 0.028623616718045704, Volatilité Perpétuelle: 0.028697684655388214

Corrélation entre les prix de clôture Spot et Perpétuel: 0.9999986054515368


Ces résultats indiquent une intégration et une efficacité élevées des marchés spot et perpétuel de Bitcoin sur Binance, avec des opportunités de gains similaires et un niveau de risque presque identique. La forte corrélation entre les deux marchés suggère également que les informations sont rapidement intégrées dans les prix sur les deux marchés, reflétant efficacement les nouvelles et les changements de sentiment du marché.

Il est important de noter que ces analyses n'ont pas encore pris en compte l'impact des coûts de financement sur les rendements des contrats perpétuels. Dans les étapes ultérieures, intégrer ces coûts sera crucial pour évaluer la rentabilité nette des positions perpétuelles par rapport aux positions spot.

